In [632]:
import random
import math
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import time

In [377]:
NUM_ALIGNMENTS = 7 #number of alignment values (0,4,16,64,256,1024,4096)
tlb = pd.read_csv('tlb.csv')
memory = pd.read_csv('Memory_Sections.csv')

In [199]:
tlb.columns

Index(['Name', 'Memory Type', 'Min Size', 'Size in KB', 'Start Address'], dtype='object')

In [200]:
memory

,Name,TLB,Size,Alignment,Start Address
0,Mem1,DDR1,3,0,NaN
1,Mem1,DDR1,4,0,NaN
2,Mem1,DDR1,5,0,NaN
3,Mem1,DDR1,1,0,NaN
4,Mem1,DDR1,2,0,NaN
5,Mem1,DDR1,2,0,NaN


In [450]:
# merge function
def Merge(a, b, order):
    c = pd.DataFrame(columns = memory.columns)
    while((len(a)!= 0) and (len(b) != 0)):
        if (b.loc[b.first_valid_index()][order] > a.loc[a.first_valid_index()][order]):
            #add b[0] to the end of c
            c = c.append(b.loc[b.first_valid_index()])
            #remove b[0] from b
            b = b.drop(b.first_valid_index())   
        else:
            # add a[0] to the end of c
            c = c.append(a.loc[a.first_valid_index()])
            #rmeove a[0] from a
            a = a.drop(a.first_valid_index())
 
    while (len(a) != 0):     
        # add a[0] to the end of c
            c = c.append(a.loc[a.first_valid_index()])
            #rmeove a[0] from a
            a = a.drop(a.first_valid_index()) 
    while (len(b) != 0):     
        # add b[0] to the end of c
            c = c.append(b.loc[b.first_valid_index()])
            #rmeove a[0] from a
            b = b.drop(b.first_valid_index())  
    return c

In [529]:
# merge function
def Merge_Backwards(a, b, order):
    c = pd.DataFrame(columns = memory.columns)
    while((len(a)!= 0) and (len(b) != 0)):
        if (b.loc[b.first_valid_index()][order] < a.loc[a.first_valid_index()][order]):
            #add b[0] to the end of c
            c = c.append(b.loc[b.first_valid_index()])
            #remove b[0] from b
            b = b.drop(b.first_valid_index())   
        else:
            # add a[0] to the end of c
            c = c.append(a.loc[a.first_valid_index()])
            #rmeove a[0] from a
            a = a.drop(a.first_valid_index())
 
    while (len(a) != 0):     
        # add a[0] to the end of c
            c = c.append(a.loc[a.first_valid_index()])
            #rmeove a[0] from a
            a = a.drop(a.first_valid_index()) 
    while (len(b) != 0):     
        # add b[0] to the end of c
            c = c.append(b.loc[b.first_valid_index()])
            #rmeove a[0] from a
            b = b.drop(b.first_valid_index())  
    return c

In [451]:
def MergeSort(mem, order):
    n = len(mem)
    if ( n == 1 ): return mem
    
    l1 = mem[0:math.floor(n/2)]
    l2 = mem[math.floor(n/2) : n]

    l1 = MergeSort( l1, order )
    l2 = MergeSort( l2, order )

    return Merge( l1, l2, order )

In [532]:
def MergeSort_Backwards(mem, order):
    n = len(mem)
    if ( n == 1 ): return mem
    
    l1 = mem[0:math.floor(n/2)]
    l2 = mem[math.floor(n/2) : n]

    l1 = MergeSort_Backwards( l1, order )
    l2 = MergeSort_Backwards( l2, order )

    return Merge_Backwards( l1, l2, order )

In [452]:
from numpy import roots
# merge function
def Merge_Alignment(a, b, order):
    # 4,16,64,256,1024,4096
    alignment_cnt = [0,0,0,0,0,0,0]
    c = pd.DataFrame(columns = memory.columns)
    while((len(a)!= 0) and (len(b) != 0)):
        if (b.loc[b.first_valid_index()][order] > a.loc[a.first_valid_index()][order]):
            #add b[0] to the end of c
            c = c.append(b.loc[b.first_valid_index()])
            alignment_cnt[int(math.log(c.loc[c.last_valid_index()][order], 4))]+=1
            #remove b[0] from b
            b = b.drop(b.first_valid_index())   
        else:
            # add a[0] to the end of c
            c = c.append(a.loc[a.first_valid_index()])
            alignment_cnt[int(math.log(c.loc[c.last_valid_index()][order], 4))]+=1
            #rmeove a[0] from a
            a = a.drop(a.first_valid_index())
 
    while (len(a) != 0):     
        # add a[0] to the end of c
            c = c.append(a.loc[a.first_valid_index()])
            alignment_cnt[int(math.log(c.loc[c.last_valid_index()][order], 4))]+=1
            #rmeove a[0] from a
            a = a.drop(a.first_valid_index()) 
    while (len(b) != 0):     
        # add b[0] to the end of c
            c = c.append(b.loc[b.first_valid_index()])
            alignment_cnt[int(math.log(c.loc[c.last_valid_index()][order], 4))]+=1
            #rmeove a[0] from a
            b = b.drop(b.first_valid_index())  
    
    return c, alignment_cnt

In [453]:
def MergeSort_Alignment(mem):
    n = len(mem)
    if ( n == 1 ): return mem
    
    l1 = mem[0:math.floor(n/2)]
    l2 = mem[math.floor(n/2) : n]

    l1 = MergeSort( l1, 'Alignment' )
    l2 = MergeSort( l2, 'Alignment' )

    return Merge_Alignment( l1, l2, 'Alignment' )

In [378]:
#mergeSort on alignment
c, alignment_cnt = MergeSort_Alignment(memory)
print(c, alignment_cnt)

B Size:  256
A Size:  256
Added:  256
Added:  256
B Size:  256
A Size:  4
Added:  256
B Size:  256
A Size:  4
Added:  256
Added:  4
B Size:  4
A Size:  4096
Added:  4096
Added:  4
B Size:  4096
A Size:  1024
Added:  4096
B Size:  4
A Size:  1024
Added:  1024
Added:  4
B Size:  4096
A Size:  256
Added:  4096
B Size:  1024
A Size:  256
Added:  1024
B Size:  4
A Size:  256
Added:  256
B Size:  4
A Size:  256
Added:  256
B Size:  4
A Size:  4
Added:  4
Added:  4
B Size:  256
A Size:  256
Added:  256
Added:  256
B Size:  256
A Size:  4
Added:  256
B Size:  256
A Size:  4
Added:  256
Added:  4
B Size:  4
A Size:  4096
Added:  4096
Added:  4
B Size:  4096
A Size:  1024
Added:  4096
B Size:  4
A Size:  1024
Added:  1024
Added:  4
B Size:  4096
A Size:  256
Added:  4096
B Size:  1024
A Size:  256
Added:  1024
B Size:  4
A Size:  256
Added:  256
B Size:  4
A Size:  256
Added:  256
B Size:  4
A Size:  4
Added:  4
Added:  4
B Size:  4096
A Size:  4096
Added:  4096
B Size:  4096
A Size:  1024
Added

In [379]:
#create a list of aligment organized arrays
df0 = pd.DataFrame(columns = memory.columns)
df1 = pd.DataFrame(columns = memory.columns)
df2 = pd.DataFrame(columns = memory.columns)
df3 = pd.DataFrame(columns = memory.columns)
df4 = pd.DataFrame(columns = memory.columns)
df5 = pd.DataFrame(columns = memory.columns)
df6 = pd.DataFrame(columns = memory.columns)

AlignmentLists = [df0,df1,df2,df3,df4,df5,df6]


In [380]:
#split into data frames by alignment
first = 0
for i in range (NUM_ALIGNMENTS):
    AlignmentLists[NUM_ALIGNMENTS-1-i] = c[first:first+alignment_cnt[NUM_ALIGNMENTS-1-i]]
    first = first + alignment_cnt[NUM_ALIGNMENTS-1-i]
    

In [381]:
print(alignment_cnt)
print(AlignmentLists[0], "\n")
print(AlignmentLists[1], "\n")
print(AlignmentLists[2], "\n")
print(AlignmentLists[3], "\n")
print(AlignmentLists[4], "\n")
print(AlignmentLists[5], "\n")
print(AlignmentLists[6], "\n")

[0, 8, 0, 0, 8, 4, 4]
Empty DataFrame
Columns: [Name, TLB, Size, Alignment, Start Address]
Index: [] 

     Name   TLB Size Alignment Start Address
0    Mem1  DDR1    3         4           NaN
5    Mem6  DDR1    2         4           NaN
6    Mem7  DDR1    2         4           NaN
11  Mem12  DDR1    5         4           NaN
12  Mem13  DDR1    5         4           NaN
17  Mem18  DDR1    2         4           NaN
18  Mem19  DDR1    1         4           NaN
23  Mem24  DDR1    5         4           NaN 

Empty DataFrame
Columns: [Name, TLB, Size, Alignment, Start Address]
Index: [] 

Empty DataFrame
Columns: [Name, TLB, Size, Alignment, Start Address]
Index: [] 

     Name   TLB Size Alignment Start Address
1    Mem2  DDR1    4       256           NaN
2    Mem3  DDR1    5       256           NaN
7    Mem8  DDR1    2       256           NaN
8    Mem9  DDR1    3       256           NaN
13  Mem14  DDR1    5       256           NaN
14  Mem15  DDR1    3       256           NaN
19  Mem20  DD

In [382]:
#sort each alignment df by size in decending order
order = 'Size'
for i in range (NUM_ALIGNMENTS):
    if(len(AlignmentLists[i])>1):
        AlignmentLists[i] = MergeSort(AlignmentLists[i], order)
        

B Size:  2
A Size:  3
Added:  3
Added:  2
B Size:  5
A Size:  2
Added:  5
Added:  2
B Size:  5
A Size:  3
Added:  5
B Size:  2
A Size:  3
Added:  3
B Size:  2
A Size:  2
Added:  2
Added:  2
B Size:  2
A Size:  5
Added:  5
Added:  2
B Size:  5
A Size:  1
Added:  5
Added:  1
B Size:  5
A Size:  5
Added:  5
B Size:  5
A Size:  2
Added:  5
B Size:  1
A Size:  2
Added:  2
Added:  1
B Size:  5
A Size:  5
Added:  5
B Size:  5
A Size:  3
Added:  5
B Size:  5
A Size:  3
Added:  5
B Size:  2
A Size:  3
Added:  3
B Size:  2
A Size:  2
Added:  2
B Size:  2
A Size:  2
Added:  2
Added:  2
Added:  1
B Size:  5
A Size:  4
Added:  5
Added:  4
B Size:  3
A Size:  2
Added:  3
Added:  2
B Size:  3
A Size:  5
Added:  5
B Size:  3
A Size:  4
Added:  4
Added:  3
Added:  2
B Size:  3
A Size:  5
Added:  5
Added:  3
B Size:  6
A Size:  4
Added:  6
Added:  4
B Size:  6
A Size:  5
Added:  6
B Size:  4
A Size:  5
Added:  5
B Size:  4
A Size:  3
Added:  4
Added:  3
B Size:  6
A Size:  5
Added:  6
B Size:  5
A Size:

In [826]:
#getNextMinAddress returns the next min address possible for the specified entry i in the current Align List
def getNextStartAddress(AlignList, i):
    if(i==0):
        #assign address to min TLB value, for now use 0
        StartAddress = 0 
    else:
        min_address = (AlignList.iloc[i-1]['Start Address'] + AlignList.iloc[i-1]['Size']+1)
        alignment = AlignList.iloc[i]['Alignment']
        StartAddress = (min_address + alignment) - (min_address + alignment)%alignment
    return StartAddress     
    

In [467]:
#assign start addresses starting from largest to smallest and merge
for i in range (NUM_ALIGNMENTS):
    if (len(AlignmentLists[NUM_ALIGNMENTS-1-i])!=0):
        first_nonzero = i
        break
for j in range(len(AlignmentLists[NUM_ALIGNMENTS-1-i])):
    AlignmentLists[NUM_ALIGNMENTS-1-i].iloc[j]['Start Address'] = getNextStartAddress(AlignmentLists[NUM_ALIGNMENTS-1-i],j)
print(AlignmentLists[NUM_ALIGNMENTS-1-i])

     Name   TLB Size Alignment Start Address
10  Mem11  DDR1    5      4096             0
22  Mem23  DDR1    5      4096          4096
16  Mem17  DDR1    3      4096          8192
4    Mem5  DDR1    2      4096         12288


In [825]:
#getNextMinAddress returns the next min address possible for the specified entry i in the current Align List
def getNextStartAddress_Merge(AlignList, i, alignment):
    if(i==0):
        #assign address to min TLB value, for now use 0
        StartAddress = 0 
    else:
        min_address = (AlignList.iloc[i]['Start Address'] + AlignList.iloc[i]['Size']+1)
        StartAddress = (min_address + alignment) - (min_address + alignment)%alignment
    return StartAddress     
    

In [815]:
# merge function
#b has addresses 
#a does not have addresses
def MergeAssignAddress(a, b, order):
    order = 'Start Address'
    c = pd.DataFrame(columns = memory.columns)
    #assign first to b
    if(len(b)>0):
        #add b[0] to the end of c
        c = c.append(b.loc[b.first_valid_index()])
        #remove b[0] from b
        b = b.drop(b.first_valid_index()) 
    while((len(a)!= 0) and (len(b) != 0)):        
        #assign a address to the earliest possible next address in c
       # a.loc[a.first_valid_index()][order] =  c.loc[-1][order] +  c.loc[-1]['Size']
        a.set_value(a.first_valid_index(), order, getNextStartAddress_Merge(c, len(c)-1, a.loc[a.first_valid_index()]['Alignment'] ))
        #a.loc[a.first_valid_index()][order] = getNextStartAddress_Merge(c, len(c)-1, a.loc[a.first_valid_index()]['Alignment'] )
        if(int(a.loc[a.first_valid_index()][order]) + int(a.loc[a.first_valid_index()]['Size']) > int(b.loc[b.first_valid_index()][order])):
            #add b[0] to the end of c
            c = c.append(b.loc[b.first_valid_index()])
            #remove b[0] from b
            b = b.drop(b.first_valid_index())   
        else:
            # add a[0] to the end of c
            c = c.append(a.loc[a.first_valid_index()])
            #rmeove a[0] from a
            a = a.drop(a.first_valid_index())

    while (len(a) != 0):     
        #assign a
        #assign a address to the earliest possible next address in c
        #a.loc[a.first_valid_index()][order] =  c.loc[-1][order] +  c.loc[-1]['Size']
        a.set_value(a.first_valid_index(),order, getNextStartAddress_Merge(c, len(c)-1, a.loc[a.first_valid_index()]['Alignment'] ))
        # add a[0] to the end of c
        c = c.append(a.loc[a.first_valid_index()])
        #rmeove a[0] from a
        a = a.drop(a.first_valid_index()) 
    while (len(b) != 0): 
        # add b[0] to the end of c
        c = c.append(b.loc[b.first_valid_index()])
        #rmeove a[0] from a
        b = b.drop(b.first_valid_index())  
    return c

In [457]:
def MergeSort_AssignAddress(mem):
    n = len(mem)
    if ( n == 1 ): return mem
    
    l1 = mem[0:math.floor(n/2)]
    l2 = mem[math.floor(n/2) : n]

    l1 = MergeSort_AssignAddress( l1, 'Start Address' )
    l2 = MergeSort_AssignAddress( l2, 'Start Address' )

    return MergeAssignAddress( l1, l2, 'Start Address' )

In [628]:
def getData_average():
    ########## AVERAGE CASE IMPORT DATA #######################
    tlbs_average =  pd.read_csv('generated_tlbs_average_case.csv')
    tlbs_average = tlbs_average.dropna()
    memory_average = pd.read_csv('generated_memory_sections_average_case.csv')
    memory_average = memory_average.dropna()
    #format data 
    tlbs_average['Start Address'] = tlbs_average['Start Address'].apply(int, base=16)
    tlbs_average['Min Size'] = tlbs_average['Min Size'].apply(int, base=16)
    tlbs_average['Size In KB'] = tlbs_average['Size In KB'].apply(str)
    tlbs_average['Size In KB'] = tlbs_average['Size In KB'].apply(int, base=16)

    memory_average['Alignment'] = memory_average['Alignment'].apply(str)
    memory_average['Alignment'] = memory_average['Alignment'].apply(int, base=16)
    memory_average['Size'] = memory_average['Size'].apply(int, base=16)
    memory = memory_average
    ################## SORT TLBS ##################
    tlb = tlbs_average
    tlb = MergeSort_Backwards(tlb, 'Start Address')
    print(tlb)
    num_tlbs = len(tlb)
    ################# Divide Memory sections into TLBs ###############
    tlb_memlist = []
    temp = pd.DataFrame()
    for i in range(num_tlbs):
        temp = (memory_average.loc[memory_average['TLB'] == tlb.iloc[i]['Name']])
        tlb_memlist.append(temp)
        # memory_average.loc[memory_average['TLB'] == 'User_TLB_DDR_3_1']
    return tlbs_average, memory_average, tlb_memlist

In [742]:
########## AVERAGE CASE IMPORT DATA #######################
tlbs_average =  pd.read_csv('generated_tlbs_average_case.csv')
tlbs_average = tlbs_average.dropna()
memory_average = pd.read_csv('generated_memory_sections_average_case.csv')
memory_average = memory_average.dropna()
#format data 
tlbs_average['Start Address'] = tlbs_average['Start Address'].apply(int, base=16)
tlbs_average['Min Size'] = tlbs_average['Min Size'].apply(int, base=16)
tlbs_average['Size In KB'] = tlbs_average['Size In KB'].apply(str)
tlbs_average['Size In KB'] = tlbs_average['Size In KB'].apply(int, base=16)

memory_average['Alignment'] = memory_average['Alignment'].apply(str)
memory_average['Alignment'] = memory_average['Alignment'].apply(int, base=16)
memory_average['Size'] = memory_average['Size'].apply(int, base=16)
memory = memory_average

In [650]:
################## SORT TLBS ##################
tlb = tlbs_average
tlb = MergeSort_Backwards(tlb, 'Start Address')
print(tlb)
num_tlbs = len(tlb)

                Name  TLB  Size  Alignment Start Address     Min Size  \
8   User_TLB_DDR_1_1  NaN   NaN        NaN             0  234109656.0   
11  User_TLB_DDR_1_4  NaN   NaN        NaN     268435456  221239593.0   
4   User_TLB_DDR_1_2  NaN   NaN        NaN     536870912  213966068.0   
0   User_TLB_DDR_1_6  NaN   NaN        NaN     805306368  130464993.0   
7   User_TLB_DDR_2_0  NaN   NaN        NaN    1073741824   55660258.0   
23  User_TLB_DDR_2_1  NaN   NaN        NaN    1140850688   30086856.0   
9   User_TLB_DDR_2_2  NaN   NaN        NaN    1207959552   22763020.0   
14  User_TLB_DDR_2_4  NaN   NaN        NaN    1275068416   18718225.0   
19  User_TLB_DDR_3_3  NaN   NaN        NaN    1342177280   14798201.0   
22  User_TLB_DDR_3_1  NaN   NaN        NaN    1358954496   13082132.0   
5   User_TLB_DDR_3_0  NaN   NaN        NaN    1375731712   10691621.0   
10  User_TLB_DDR_3_2  NaN   NaN        NaN    1392508928    4956311.0   
16  User_TLB_DDR_4_3  NaN   NaN        NaN    14092

In [743]:
################# Divide Memory sections into TLBs ###############
tlb_memlist = []
temp = pd.DataFrame()
for i in range(num_tlbs):
    temp = (memory_average.loc[memory_average['TLB'] == tlb.iloc[i]['Name']])
    tlb_memlist.append(temp)
    # memory_average.loc[memory_average['TLB'] == 'User_TLB_DDR_3_1']

In [640]:
import time
def runMergeSort(memory):

    #create a list of aligment organized arrays
    df0 = pd.DataFrame(columns = memory.columns)
    df1 = pd.DataFrame(columns = memory.columns)
    df2 = pd.DataFrame(columns = memory.columns)
    df3 = pd.DataFrame(columns = memory.columns)
    df4 = pd.DataFrame(columns = memory.columns)
    df5 = pd.DataFrame(columns = memory.columns)
    df6 = pd.DataFrame(columns = memory.columns)

    AlignmentLists = [df0,df1,df2,df3,df4,df5,df6]
    st = time.perf_counter()
    #startTime = time.perf_counter()
    ############################################ START TIMING ####################################
    #mergeSort on alignment
    c, alignment_cnt = MergeSort_Alignment(memory)

    #split into data frames by alignment
    first = 0
    for i in range (NUM_ALIGNMENTS):
        AlignmentLists[NUM_ALIGNMENTS-1-i] = c[first:first+alignment_cnt[NUM_ALIGNMENTS-1-i]]
        first = first + alignment_cnt[NUM_ALIGNMENTS-1-i]

    #sort each alignment df by size in decending order
    order = 'Size'
    for i in range (NUM_ALIGNMENTS):
        if(len(AlignmentLists[i])>1):
            AlignmentLists[i] = MergeSort(AlignmentLists[i], order)    

    #assign start addresses starting from largest to smallest and merge
    for i in range (NUM_ALIGNMENTS):
        if (len(AlignmentLists[NUM_ALIGNMENTS-1-i])!=0):
            first_nonzero = i
            break
    for j in range(len(AlignmentLists[NUM_ALIGNMENTS-1-i])):
        AlignmentLists[NUM_ALIGNMENTS-1-i].set_value(AlignmentLists[NUM_ALIGNMENTS-1-i].index[j], 'Start Address',getNextStartAddress(AlignmentLists[NUM_ALIGNMENTS-1-i],j))

    #merge all
    b = AlignmentLists[NUM_ALIGNMENTS-1-first_nonzero]
    for j in range(first_nonzero+1,NUM_ALIGNMENTS):
        a = AlignmentLists[NUM_ALIGNMENTS-1-j-first_nonzero]
        b = MergeAssignAddress(a, b, 'Start Address')
    ############################################ END TIMING ####################################
    et = time.perf_counter()
    totalTime = et-st
    return b, totalTime

In [762]:
runtime = 0
tlb_average = pd.DataFrame()
memory_average = pd.DataFrame()
tlb_memlist = []
tlb_average, memory_average, tlb_memlist =  getData_average()
memory = memory_average
#b, runtime = runMergeSort(memory_average)
#print(runtime)
#print(b)

                Name  TLB  Size  Alignment Start Address     Min Size  \
8   User_TLB_DDR_1_1  NaN   NaN        NaN             0  234109656.0   
11  User_TLB_DDR_1_4  NaN   NaN        NaN     268435456  221239593.0   
4   User_TLB_DDR_1_2  NaN   NaN        NaN     536870912  213966068.0   
0   User_TLB_DDR_1_6  NaN   NaN        NaN     805306368  130464993.0   
7   User_TLB_DDR_2_0  NaN   NaN        NaN    1073741824   55660258.0   
23  User_TLB_DDR_2_1  NaN   NaN        NaN    1140850688   30086856.0   
9   User_TLB_DDR_2_2  NaN   NaN        NaN    1207959552   22763020.0   
14  User_TLB_DDR_2_4  NaN   NaN        NaN    1275068416   18718225.0   
19  User_TLB_DDR_3_3  NaN   NaN        NaN    1342177280   14798201.0   
22  User_TLB_DDR_3_1  NaN   NaN        NaN    1358954496   13082132.0   
5   User_TLB_DDR_3_0  NaN   NaN        NaN    1375731712   10691621.0   
10  User_TLB_DDR_3_2  NaN   NaN        NaN    1392508928    4956311.0   
16  User_TLB_DDR_4_3  NaN   NaN        NaN    14092

START USING built in functions

In [ ]:
import time
def runMergeSort(memory):

    #create a list of aligment organized arrays
    df0 = pd.DataFrame(columns = memory.columns)
    df1 = pd.DataFrame(columns = memory.columns)
    df2 = pd.DataFrame(columns = memory.columns)
    df3 = pd.DataFrame(columns = memory.columns)
    df4 = pd.DataFrame(columns = memory.columns)
    df5 = pd.DataFrame(columns = memory.columns)
    df6 = pd.DataFrame(columns = memory.columns)

    AlignmentLists = [df0,df1,df2,df3,df4,df5,df6]
    st = time.perf_counter()
    #startTime = time.perf_counter()
    ############################################ START TIMING ####################################
    #mergeSort on alignment
    c, alignment_cnt = MergeSort_Alignment(memory)

    #split into data frames by alignment
    first = 0
    for i in range (NUM_ALIGNMENTS):
        AlignmentLists[NUM_ALIGNMENTS-1-i] = c[first:first+alignment_cnt[NUM_ALIGNMENTS-1-i]]
        first = first + alignment_cnt[NUM_ALIGNMENTS-1-i]

    #sort each alignment df by size in decending order
    order = 'Size'
    for i in range (NUM_ALIGNMENTS):
        if(len(AlignmentLists[i])>1):
            AlignmentLists[i] = MergeSort(AlignmentLists[i], order)    

    #assign start addresses starting from largest to smallest and merge
    for i in range (NUM_ALIGNMENTS):
        if (len(AlignmentLists[NUM_ALIGNMENTS-1-i])!=0):
            first_nonzero = i
            break
    for j in range(len(AlignmentLists[NUM_ALIGNMENTS-1-i])):
        AlignmentLists[NUM_ALIGNMENTS-1-i].set_value(AlignmentLists[NUM_ALIGNMENTS-1-i].index[j], 'Start Address',getNextStartAddress(AlignmentLists[NUM_ALIGNMENTS-1-i],j))

    #merge all
    b = AlignmentLists[NUM_ALIGNMENTS-1-first_nonzero]
    for j in range(first_nonzero+1,NUM_ALIGNMENTS):
        a = AlignmentLists[NUM_ALIGNMENTS-1-j-first_nonzero]
        b = MergeAssignAddress(a, b, 'Start Address')
    ############################################ END TIMING ####################################
    et = time.perf_counter()
    totalTime = et-st
    return b, totalTime

In [844]:
#IMPORT
runtime = 0
tlb_average = pd.DataFrame()
memory_average = pd.DataFrame()
tlb_memlist = []
tlb_average, memory_average, tlb_memlist =  getData_average()
memory = memory_average
tlb = tlb_average

                Name  TLB  Size  Alignment Start Address     Min Size  \
8   User_TLB_DDR_1_1  NaN   NaN        NaN             0  234109656.0   
11  User_TLB_DDR_1_4  NaN   NaN        NaN     268435456  221239593.0   
4   User_TLB_DDR_1_2  NaN   NaN        NaN     536870912  213966068.0   
0   User_TLB_DDR_1_6  NaN   NaN        NaN     805306368  130464993.0   
7   User_TLB_DDR_2_0  NaN   NaN        NaN    1073741824   55660258.0   
23  User_TLB_DDR_2_1  NaN   NaN        NaN    1140850688   30086856.0   
9   User_TLB_DDR_2_2  NaN   NaN        NaN    1207959552   22763020.0   
14  User_TLB_DDR_2_4  NaN   NaN        NaN    1275068416   18718225.0   
19  User_TLB_DDR_3_3  NaN   NaN        NaN    1342177280   14798201.0   
22  User_TLB_DDR_3_1  NaN   NaN        NaN    1358954496   13082132.0   
5   User_TLB_DDR_3_0  NaN   NaN        NaN    1375731712   10691621.0   
10  User_TLB_DDR_3_2  NaN   NaN        NaN    1392508928    4956311.0   
16  User_TLB_DDR_4_3  NaN   NaN        NaN    14092

In [797]:
memory = tlb_memlist[0].copy()
memory['Start Address'].values[:] = 0
print(memory)

                                   Name               TLB      Size  \
9     Memory_Section_User_TLB_DDR_1_1_0  User_TLB_DDR_1_1   1742593   
26   Memory_Section_User_TLB_DDR_1_1_19  User_TLB_DDR_1_1    296548   
32   Memory_Section_User_TLB_DDR_1_1_13  User_TLB_DDR_1_1   6652372   
78   Memory_Section_User_TLB_DDR_1_1_28  User_TLB_DDR_1_1   9047884   
81    Memory_Section_User_TLB_DDR_1_1_8  User_TLB_DDR_1_1   7357313   
114   Memory_Section_User_TLB_DDR_1_1_4  User_TLB_DDR_1_1  12563712   
118  Memory_Section_User_TLB_DDR_1_1_12  User_TLB_DDR_1_1  11670948   
144   Memory_Section_User_TLB_DDR_1_1_7  User_TLB_DDR_1_1   6900115   
148  Memory_Section_User_TLB_DDR_1_1_35  User_TLB_DDR_1_1  12738253   
161  Memory_Section_User_TLB_DDR_1_1_30  User_TLB_DDR_1_1   6611805   
175   Memory_Section_User_TLB_DDR_1_1_3  User_TLB_DDR_1_1  12832270   
253  Memory_Section_User_TLB_DDR_1_1_20  User_TLB_DDR_1_1  13122990   
265  Memory_Section_User_TLB_DDR_1_1_22  User_TLB_DDR_1_1    400246   
285  M

In [846]:
def main_average_case():
     #########################
    #IMPORT
    runtime = 0
    tlb_average = pd.DataFrame()
    memory_average = pd.DataFrame()
    tlb_memlist = []
    tlb_average, memory_average, tlb_memlist =  getData_average()
    memory = memory_average
    tlb = tlb_average
    ###############################
    #assign memory 
    memory = tlb_memlist[0].copy()
    memory['Start Address'].values[:] = 0
    ###############################
    mem_map = pd.DataFrame(columns = memory.columns)
    #for each TLB, sort
    for i in range(len(tlb_memlist)):
        #get the TLB start address
        tlb_start = 
        memory = tlb_memlist[i].copy()
        memory['Start Address'].values[:] = 0 #clear start addresses

        #create a list of aligment organized arrays
        df0 = pd.DataFrame(columns = memory.columns)
        df1 = pd.DataFrame(columns = memory.columns)
        df2 = pd.DataFrame(columns = memory.columns)
        df3 = pd.DataFrame(columns = memory.columns)
        df4 = pd.DataFrame(columns = memory.columns)
        df5 = pd.DataFrame(columns = memory.columns)
        df6 = pd.DataFrame(columns = memory.columns)

        AlignmentLists = [df0,df1,df2,df3,df4,df5,df6]

        #count the number of 
        #(0,4,16,64,256,1024,4096) 
        alignment_cnt = [0,0,0,0,0,0,0]
        for i in range(len(memory)):
            alignment_cnt[int(math.log(memory.iloc[i]['Alignment'],4))]+=1  
        b=MergeSortTest(memory)
        mem_map = mem_map.append(b)

SyntaxError: invalid syntax (<ipython-input-846-c156e9c1b8fd>, line 5)

In [843]:
print(mem_map)

                                   Name               TLB      Size Alignment  \
517   Memory_Section_User_TLB_DDR_1_1_2  User_TLB_DDR_1_1  11677145      4096   
411  Memory_Section_User_TLB_DDR_1_1_32  User_TLB_DDR_1_1  11612645       256   
118  Memory_Section_User_TLB_DDR_1_1_12  User_TLB_DDR_1_1  11670948      4096   
319  Memory_Section_User_TLB_DDR_1_1_11  User_TLB_DDR_1_1  11269710      4096   
894  Memory_Section_User_TLB_DDR_1_1_27  User_TLB_DDR_1_1    653649      4096   
253  Memory_Section_User_TLB_DDR_1_1_20  User_TLB_DDR_1_1  13122990      1024   
114   Memory_Section_User_TLB_DDR_1_1_4  User_TLB_DDR_1_1  12563712      1024   
447  Memory_Section_User_TLB_DDR_1_1_10  User_TLB_DDR_1_1   4403823      1024   
647   Memory_Section_User_TLB_DDR_1_1_1  User_TLB_DDR_1_1  11360052       256   
695  Memory_Section_User_TLB_DDR_1_1_26  User_TLB_DDR_1_1  10207438       256   
552  Memory_Section_User_TLB_DDR_1_1_15  User_TLB_DDR_1_1   7850559       256   
144   Memory_Section_User_TL

In [835]:
# at the top of the file, before other imports
import warnings

warnings.filterwarnings('ignore')

def MergeSortTest(memory):
    #sort by alignment
    c = pd.DataFrame(columns = memory.columns)
    memory.sort_values(['Alignment'], axis = 0, ascending = False, inplace=True, kind='mergesort')
    c = memory.copy()
    

    #split into data frames by alignment
    first = 0
    for i in range (NUM_ALIGNMENTS):
        AlignmentLists[NUM_ALIGNMENTS-1-i] = c[first:first+alignment_cnt[NUM_ALIGNMENTS-1-i]]
        first = first + alignment_cnt[NUM_ALIGNMENTS-1-i]
     #sort by size
    #sort each alignment df by size in decending order
    order = 'Size'
    for i in range (NUM_ALIGNMENTS):
        if(len(AlignmentLists[i])>1):
            #AlignmentLists[i] = MergeSort(AlignmentLists[i], order)    
            AlignmentLists[i].sort_values(['Size'], axis = 0, ascending = False, inplace=True, kind='mergesort')  
    
    #assign start addresses starting from largest to smallest and merge
    for i in range (NUM_ALIGNMENTS):
        if (len(AlignmentLists[NUM_ALIGNMENTS-1-i])!=0):
            first_nonzero = i
            break
    for j in range(len(AlignmentLists[NUM_ALIGNMENTS-1-i])):
        AlignmentLists[NUM_ALIGNMENTS-1-i].set_value(AlignmentLists[NUM_ALIGNMENTS-1-i].index[j], 'Start Address',getNextStartAddress(AlignmentLists[NUM_ALIGNMENTS-1-i],j))

    #merge all
    b = AlignmentLists[NUM_ALIGNMENTS-1-first_nonzero]
    for j in range(first_nonzero+1,NUM_ALIGNMENTS):
        a = AlignmentLists[NUM_ALIGNMENTS-1-j-first_nonzero]
        b = MergeAssignAddress(a, b, 'Start Address')
    
    return b